# Analisis de superficial del csv de clicks
### En este notebook exploramos los registros del csv con el objetivo de buscar relaciones entre las distintas variables

In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Leemos el csv, y dropeamos las columnas action_id por estar llena de nans y wifi_connection por estar llena de False

In [2]:
installs = pd.read_csv("../data/installs.csv")
installs['created'] = pd.to_datetime(installs['created'], errors = 'coerce')
installs.head(5)

/home/gonzalo/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,6287817205707153877,NaN,3.739127e+17,adjust.com,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN,4243443387795468703,3.301378e+18
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,6287817205707153877,NaN,7.805539e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4724288679627032761,3.301378e+18
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355496e+18,adjust.com,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060930e+18
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.355772e+18,adjust.com,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.301378e+18
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.301378e+18


In [3]:
installs = installs.sort_values(by = 'created')

In [4]:
installs = installs[['ref_hash', 'created']]

In [5]:
fecha_minima=pd.to_datetime('2019-04-18 00:00:00')
fecha_tope=pd.to_datetime('2019-04-21 00:00:00')
installs_ventana1=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [6]:
fecha_minima=pd.to_datetime('2019-04-19 00:00:00')
fecha_tope=pd.to_datetime('2019-04-22 00:00:00')
installs_ventana2=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [7]:
fecha_minima=pd.to_datetime('2019-04-20 00:00:00')
fecha_tope=pd.to_datetime('2019-04-23 00:00:00')
installs_ventana3=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [8]:
fecha_minima=pd.to_datetime('2019-04-21 00:00:00')
fecha_tope=pd.to_datetime('2019-04-24 00:00:00')
installs_ventana4=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [9]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00')
fecha_tope=pd.to_datetime('2019-04-27 00:00:00')
installs_ventana7=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)

In [10]:
features = ['sum', 'mean', 'std', 'min', 'max']
ventanas = [installs_ventana1, installs_ventana2, installs_ventana3, installs_ventana4, installs_ventana7]
numero_de_ventanas = [1,2,3,4,7]
contador = 0


In [11]:
for ventana in ventanas:
    ventana['time_diff'] = ventana.groupby('ref_hash').agg('diff')/ np.timedelta64(1, 's')
    ventana_aux = ventana.groupby('ref_hash')['time_diff'].agg('sum').to_frame()
    ventana_aux['time_diff_mean'] = ventana.groupby('ref_hash')['time_diff'].agg('mean')
    ventana_aux['time_diff_std'] = ventana.groupby('ref_hash')['time_diff'].agg('std')
    ventana_aux['time_diff_max'] = ventana.groupby('ref_hash')['time_diff'].agg('max')
    ventana_aux['time_diff_min'] = ventana.groupby('ref_hash')['time_diff'].agg('min')
    ventana_aux = ventana_aux.rename(columns = {'time_diff':'time_diff_sum'})
    ventana_aux = ventana_aux.fillna(259200)
    ventana_aux['time_diff_sum'] = (ventana_aux['time_diff_sum'] == 0) * 259200 + ventana_aux['time_diff_sum']
    ventana_aux.to_csv('../xgb/features_timediff_installs_ventana' + str(numero_de_ventanas[contador]) + '.csv')
    contador = contador + 1